In [120]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import time
from dateutil.relativedelta import relativedelta

%matplotlib inline
import matplotlib
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
# matplotlib.style.use('seaborn-paper')
import matplotlib.pyplot as plt

In [149]:
import os
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score


def load_data():
    # Load the dataset
    os.chdir("../indicator_data/")
    data = pd.read_csv("dataset_basis.csv")
    length = len(data)
    split_point = int(0.95 * length)

    # Split the dataset into training and testing sets
    input_train = data.iloc[0:split_point, 1:2]
    input_test = data.iloc[split_point:, 1:2]
    output_train = data.iloc[0:split_point, 2]
    output_test = data.iloc[split_point:, 2]

    # Convert the data to DMatrix format
    dtrain = xgb.DMatrix(input_train.values, label=output_train.values)
    dtest = xgb.DMatrix(input_test.values, label=output_test.values)

    return dtrain, dtest, output_train, output_test


def train_model(dataset_train):
    # Set the hyperparameters for XGBoost
    # params = {
    #     'max_depth': 5,
    #     'objective': "multi:softmax",
    #     'num_class': 2,
    #     'subsample': 0.7
    # }

    params = {
        'max_depth': 7,
        'objective': "multi:softmax",
        'num_class': 2,
        'colsample_bytree': 0.6,
        'learning_rate': 0.3,
        'subsample': 0.6
    }

    # Train the model using XGBoost
    num_rounds = 30
    xgb_model = xgb.train(params, dataset_train, num_boost_round=num_rounds)

    return xgb_model


def evaluate_model(trained_model, dataset_train, dataset_test, output_train, output_test):
    # Make predictions on the training and test data
    y_train_pred = trained_model.predict(dataset_train)
    y_test_pred = trained_model.predict(dataset_test)

    # Calculate the accuracy scores
    train_accuracy = accuracy_score(output_train, y_train_pred)
    test_accuracy = accuracy_score(output_test, y_test_pred)

    # Print the accuracy scores
    print('Training Accuracy: %.2f%%' % (train_accuracy * 100.0))
    print('Testing Accuracy: %.2f%%' % (test_accuracy * 100.0))

    return y_train_pred, y_test_pred


if __name__ == "__main__":
    # Load the data
    trainSet, testSet, y_train, y_test = load_data()

    # Train the model
    model = train_model(trainSet)

    # Evaluate the model and save the signals
    train_signal, test_signal = evaluate_model(model, trainSet, testSet, y_train, y_test)


Training Accuracy: 69.80%
Testing Accuracy: 51.06%


In [217]:
pd.set_option('display.max_rows', 10)
os.chdir("../indicator_data/")
data = pd.read_csv("signal_basis.csv")
data

,Unnamed: 0,date,basis_indicator,y,signal
0,0,2007-11-06,-1586.0,1,0.0
1,1,2007-11-07,-1587.0,0,0.0
2,2,2007-11-08,-1857.0,0,0.0
3,3,2007-11-09,-1732.0,0,0.0
4,4,2007-11-12,-1662.0,0,0.0
...,...,...,...,...,...
3739,3739,2023-03-24,1243.0,1,1.0
3740,3740,2023-03-27,1319.0,1,1.0
3741,3741,2023-03-28,1271.0,1,0.0
3742,3742,2023-03-29,986.0,0,1.0


In [218]:
# load the signal
os.chdir("../indicator_data/")
data = pd.read_csv("signal_basis.csv")
split_point = int(len(data) * 0.95)

# load the price data
os.chdir("../raw_data/")
df_future_price = pd.read_csv("CF_price.csv")
df_future_price = df_future_price[df_future_price['CLOCK'] >= "2007-11-06"]




# compute action and position based on signal 
signal = data[['date', 'signal']]
df = df_future_price[['CLOCK', 'CLOSE']].merge(signal, left_on="CLOCK", right_on="date").drop(columns="date")
df.columns = ['date', 'close_price', 'signal']

# if there's consecutive long and short, filter it out (optional)
df['position'] = df['signal'].diff().fillna(0)
df['position_filter'] = df['position'].diff().fillna(0)

index_list_1 = df[df['position_filter'] == -2].index.values - 1
index_list_2 = df[df['position_filter'] == 2].index.values - 1

df['signal_filter'] = df['signal']

df.loc[index_list_1.tolist(), 'signal_filter'] = 0
df.loc[index_list_2.tolist(), 'signal_filter'] = 1

df['position_2'] = df['signal_filter'].diff().fillna(0)
df.loc[0, 'position_2'] = -1

# result
df_result = df[['date', 'close_price', 'signal_filter', 'position_2']]
df_result.columns = ['date', 'close_price', 'signal', 'action']
df_result['position'] = df_result['action'].replace(0, np.nan).fillna(method='ffill').fillna(0)

df_result_train = df_result.iloc[0:split_point]
df_result_test = df_result.iloc[split_point:]

In [220]:
df_result_train

,date,close_price,signal,action,position
0,2007-11-06,15135.0,0.0,-1.0,-1.0
1,2007-11-07,15390.0,0.0,0.0,-1.0
2,2007-11-08,15265.0,0.0,0.0,-1.0
3,2007-11-09,15195.0,0.0,0.0,-1.0
4,2007-11-12,15140.0,0.0,0.0,-1.0
...,...,...,...,...,...
3551,2022-06-16,19560.0,0.0,0.0,-1.0
3552,2022-06-17,19515.0,1.0,1.0,1.0
3553,2022-06-20,19240.0,0.0,-1.0,-1.0
3554,2022-06-21,19065.0,0.0,0.0,-1.0


In [ ]:
# return in train set

# daily return 
df_result_train['daily_return'] = df_result_train['close_price'].pct_change() * df_result_train['position']
# cum return
df_result_train['cumulative_return'] = (1 + df_result_train['daily_return']).cumprod() - 1
# net value
df_result_train['daily_net_return'] = (1 + df_result_train['daily_return']).cumprod() - 1
df_result_train['net_value'] = 1 * (1 + df_result_train['daily_net_return'])
# max draw down
Roll_Max = df_result_train['net_value'].cummax()
Daily_Drawdown = df_result_train['net_value']/Roll_Max - 1.0
Max_Daily_Drawdown = (Daily_Drawdown.cummin().iloc[-1] * 100)
# avg and std
avg = df_result_train['daily_return'].mean()
std = df_result_train['daily_return'].std()
sharpe = np.sqrt(365) * (avg / std)
calmar = 




# return in test set
df_result_test['daily_return'] = df_result_test['close_price'].pct_change() * df_result_test['position']
df_result_test['cumulative_return'] = (1 + df_result_test['daily_return']).cumprod() - 1

In [172]:
# transacion times and time period
start_date_train = datetime.datetime.strptime(df_result_train['date'].iloc[0], "%Y-%m-%d").date()
end_date_train = datetime.datetime.strptime(df_result_train['date'].iloc[-1], "%Y-%m-%d").date()
time_delta_train = relativedelta(end_date_train, start_date_train).years

transaction_time_train = len(df_result_train[df_result_train['action'] != 0])
transaction_time_average_train = transaction_time_train / time_delta_train

print(start_date_train, end_date_train, time_delta_train, transaction_time_train, transaction_time_average_train)

start_date_test = datetime.datetime.strptime(df_result_test['date'].iloc[0], "%Y-%m-%d").date()
end_date_test = datetime.datetime.strptime(df_result_test['date'].iloc[-1], "%Y-%m-%d").date()
time_delta_test = relativedelta(end_date_test, start_date_test).months / 12

transaction_time_test = len(df_result_test[df_result_test['action'] != 0])
transaction_time_average_test = transaction_time_test / time_delta_test
print(start_date_test, end_date_test, time_delta_test, transaction_time_test, transaction_time_average_test)

2007-11-06 2022-06-22 14 775 55.357142857142854
2022-06-23 2023-03-30 0.75 44 58.666666666666664


In [173]:
# return and max drawdown
cum_return_train = df_result_train['cumulative_return'].iloc[-1]
# maximum drawdown



cum_return_test = df_result_test['cumulative_return'].iloc[-1]


In [206]:
df_result_train.head(40)

,date,close_price,signal,action,position,daily_return,cumulative_return,daily_net_return,net_value
0,2007-11-06,15135.0,0.0,-1.0,-1.0,NaN,NaN,NaN,NaN
1,2007-11-07,15390.0,0.0,0.0,-1.0,-0.016848,-0.016848,-0.016848,0.983152
2,2007-11-08,15265.0,0.0,0.0,-1.0,0.008122,-0.008863,-0.008863,0.991137
3,2007-11-09,15195.0,0.0,0.0,-1.0,0.004586,-0.004318,-0.004318,0.995682
4,2007-11-12,15140.0,0.0,0.0,-1.0,0.003620,-0.000714,-0.000714,0.999286
5,2007-11-13,14885.0,0.0,0.0,-1.0,0.016843,0.016117,0.016117,1.016117
6,2007-11-14,14920.0,0.0,0.0,-1.0,-0.002351,0.013727,0.013727,1.013727
7,2007-11-15,14840.0,0.0,0.0,-1.0,0.005362,0.019163,0.019163,1.019163
8,2007-11-16,14815.0,0.0,0.0,-1.0,0.001685,0.020880,0.020880,1.020880
9,2007-11-19,14870.0,0.0,0.0,-1.0,-0.003712,0.017090,0.017090,1.017090


In [204]:
fig, ax = plt.subplots(dpi=1000)
plt.rcParams['figure.figsize'] = [14, 4]
plt.rcParams["font.size"] = 15
plt.plot(df_result_train['date'], df_result_train['net_value'])

plt.xlabel("date")
plt.ylabel("net value (unit: %)")

plt.xticks(rotation=45)
ax.set_xticks([x for x in range(0,len(df_result_train['date']),180)])

plt.title('net value (train set)')
plt.show()

In [ ]:
# return in test set
pd.set_option('display.max_rows', 10)

df_result_test['daily_return'] = df_result_test['close_price'].pct_change() * df_result_test['position']
df_result_test['cumulative_return'] = (1 + df_result_test['daily_return']).cumprod() - 1
df_result_test

,date,close_price,signal,action,position,daily_return,cumulative_return
3556,2022-06-23,18435.0,1.0,1.0,1.0,NaN,NaN
3557,2022-06-24,17230.0,1.0,0.0,1.0,-0.065365,-0.065365
3558,2022-06-27,17145.0,1.0,0.0,1.0,-0.004933,-0.069976
3559,2022-06-28,17615.0,1.0,0.0,1.0,0.027413,-0.044481
3560,2022-06-29,17575.0,1.0,0.0,1.0,-0.002271,-0.046650
...,...,...,...,...,...,...,...
3739,2023-03-24,13895.0,1.0,0.0,1.0,-0.005013,-0.126329
3740,2023-03-27,13910.0,0.0,-1.0,-1.0,-0.001080,-0.127273
3741,2023-03-28,14205.0,1.0,1.0,1.0,0.021208,-0.108764
3742,2023-03-29,14380.0,0.0,-1.0,-1.0,-0.012320,-0.119744


In [201]:
pd.set_option('display.max_rows', None)
df_result_train.head(20)

,date,close_price,signal,action,position,daily_return,cumulative_return,daily_net_return,net_value
0,2007-11-06,15135.0,0.0,-1.0,-1.0,NaN,NaN,NaN,NaN
1,2007-11-07,15390.0,0.0,0.0,-1.0,-0.016848,-0.016848,-0.016848,0.983152
2,2007-11-08,15265.0,0.0,0.0,-1.0,0.008122,-0.008863,-0.008863,0.991137
3,2007-11-09,15195.0,0.0,0.0,-1.0,0.004586,-0.004318,-0.004318,0.995682
4,2007-11-12,15140.0,0.0,0.0,-1.0,0.003620,-0.000714,-0.000714,0.999286
5,2007-11-13,14885.0,0.0,0.0,-1.0,0.016843,0.016117,0.016117,1.016117
6,2007-11-14,14920.0,0.0,0.0,-1.0,-0.002351,0.013727,0.013727,1.013727
7,2007-11-15,14840.0,0.0,0.0,-1.0,0.005362,0.019163,0.019163,1.019163
8,2007-11-16,14815.0,0.0,0.0,-1.0,0.001685,0.020880,0.020880,1.020880
9,2007-11-19,14870.0,0.0,0.0,-1.0,-0.003712,0.017090,0.017090,1.017090


In [118]:
os.chdir("../raw_data/")
df_future_price = pd.read_csv("CF_price.csv")
df_future_price = df_future_price[df_future_price['CLOCK'] >= "2007-11-06"]

# visualize future close price

fig, ax = plt.subplots(dpi=1000)

plt.rcParams['figure.figsize'] = [14, 4]
plt.plot(df_future_price['CLOCK'], df_future_price['CLOSE'])

plt.xlabel("date")
plt.ylabel("close price (continuous)")

plt.xticks(rotation=45)
ax.set_xticks([x for x in range(0,len(df_future_price['CLOCK']),180)])

plt.title('Long position (red) and short position (green)')

y1 = np.arange(0, 1, 0.01)
y2 = np.arange(-1, 0, 0.01)
ax2 = ax.twinx()
for i in range(len(df)):
    row = df.iloc[i]
    temp_date = row['date']
    if row['signal_filter'] == 1:
        # plt.axvline(x=temp_date, color='red', alpha=.1)
        ax2.fill_betweenx(y1, temp_date, temp_date,  color='red', alpha=.1)
    elif row['signal_filter'] == 0:
        # plt.axvline(x=temp_date, color='green', alpha=.1)
        ax2.fill_betweenx(y2, temp_date, temp_date,  color='green', alpha=.1)

plt.show()